In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
from networkx.utils import py_random_state
from networkx.generators.random_graphs import _random_subset
from networkx.generators.classic import star_graph
import numpy as np
import random

import scipy.stats as stats
from scipy.optimize import curve_fit


import time
import os

In [ ]:
seed = np.random.RandomState()           
time_steps = int(1e5) 
# t0 = time.time()

explorations_per_graph=3

def LinearSearch(lys, element):
    for i in range(len(lys)):
        if lys[i] == element:
            return i
    return -1

#definisco una funzione del grafo per poi fare la media tra grafi differenti

def time_nodes (G, explorations_per_graph):
    initial_node = _random_subset(G.nodes, 1, seed).pop()
       
    #dinamica di esplorazione: semplicemente riempio una lista dei nuovi nodi in cui va il walker. Linear search 
    #mi guarda se il nodo è gia presente e non lo aggiunge. Exploration fa un evoluzione temporale restituendo il nuovo nodo
    
    def exploration(initial_node, G, explored_nodes):
        if LinearSearch(explored_nodes, initial_node) < 0:
               explored_nodes += [initial_node]
        neighbours = list(G[initial_node])
        new_node = _random_subset(neighbours, 1, seed).pop()
        return new_node

    exploration_time = []
    explored_nodes = []

    #ripeto l'esplorazione di "time_steps" passi temporali "exploration_per_graph" volte per fare una media - i tempi 
    #di esplorazione variano molto di volta in volta. Semplicemente ogni iterazione mi cambia il nodo iniziale con quello
    #nuovo chiamando la funzione esplorazione. Quando tutti i nodi sono esplorati salvo il tempo impiegato e svuoto il vettore
    #Infine faccio una media dei tempi.
    
    for i in range(explorations_per_graph):
        t = 0
        while True:
            initial_node = exploration(initial_node, G, explored_nodes)
            if len(explored_nodes) == len(G.nodes):
                exploration_time += [t]
                explored_nodes.clear()
                break
            t += 1

    mean_exploration_time = sum(exploration_time)/explorations_per_graph
    print("mean_exploration_time: ", mean_exploration_time, "for", len(G.nodes) ,"nodes")

    return mean_exploration_time
    
#print(exploration_time)
#print(explored_nodes) 
#print(len(explored_nodes),"Explored nodes")
     

# print(f"done in {int((time.time()-t0)/60)} minutes and {((time.time()-t0)%60)} seconds")

In [5]:
Times = []
N = [200+200*i for i in range(30)]

for i in range(len(N)):
    G = nx.complete_graph(N[i])
    Times += [time_nodes(G,explorations_per_graph)]

mean_exploration_time:  1152.74 for 200 nodes
mean_exploration_time:  2673.9 for 400 nodes
mean_exploration_time:  4131.52 for 600 nodes
mean_exploration_time:  5528.66 for 800 nodes
mean_exploration_time:  7452.72 for 1000 nodes
mean_exploration_time:  9118.68 for 1200 nodes
mean_exploration_time:  10883.54 for 1400 nodes
mean_exploration_time:  13025.42 for 1600 nodes
mean_exploration_time:  15049.7 for 1800 nodes
mean_exploration_time:  15947.2 for 2000 nodes
mean_exploration_time:  17779.12 for 2200 nodes
mean_exploration_time:  20323.36 for 2400 nodes
mean_exploration_time:  22169.28 for 2600 nodes
mean_exploration_time:  24434.42 for 2800 nodes
mean_exploration_time:  25625.28 for 3000 nodes
mean_exploration_time:  27268.98 for 3200 nodes
mean_exploration_time:  30389.94 for 3400 nodes
mean_exploration_time:  32215.58 for 3600 nodes
mean_exploration_time:  33650.58 for 3800 nodes
mean_exploration_time:  34423.72 for 4000 nodes
mean_exploration_time:  37552.4 for 4200 nodes
mean_e

In [1]:
fig, ax = plt.subplots()
    
ax.plot(N, Times, '.', label = 'exploration_time')
ax.set_xlabel("Number of nodes")
ax.set_ylabel("Exploration time")

plt.show()

linear_fit=np.polyfit(N,Times,1)

print(linear_fit)

fit = [linear_fit[0]*n+linear_fit[1] for n in N]
fig, ax = plt.subplots()
    
ax.plot(N, Times, '.', label = f"exploration time")
ax.set_xlabel("Number of nodes")
ax.set_ylabel(f"Exploration time")

ax.plot(N, fit, '-', label = f"fit: {round(linear_fit[1],4)} + x • {round(linear_fit[0],3)}")

n_logn = [1.05*n*np.log(n) for n in N]

ax.plot(N, n_logn, '-', label = f"fit:)

ax.legend()

plt.show()



SyntaxError: EOL while scanning string literal (<ipython-input-1-149780a7c5f4>, line 24)